Using the Fusion API provided by Yelp, scrape data from restaurants in New York City

In [22]:
import pandas as pd 
import numpy as np
import requests
from yelpapi import YelpAPI
import time 
from urllib.parse import quote


In [23]:
client_id = '-nR8bKzJL-ilvjiSSbgTgw'
api_key =  'fP7j0wJuOHXq3FG6Y7flI1OD9r4yWnbIGbx6yce2Z42C64K6WhupmUR9O3SvZNW6nmQVzjfeASxV_liT6MLttZlFdIFtch32PCCnVs0d8Sd0X3sSPTCkNZyma7tdY3Yx'

In [28]:
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'

def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location, limit, offset):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """


    url_params = {'term': term, 'location': location, 'limit': limit, 'offset': offset}

    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

In [29]:
term = 'restaurant'
location = 'New York City'
limit = 50


dfs_lst = []

offset = 0
for i in range(19):
    response = search(api_key, term, location, limit, offset)
    start = time.time()
    # Name, category, coordinates, price, zipcode
    businesses = response['businesses']

    rest_names_lst = []
    category_lst = []
    lat_lst = []
    lon_lst = []
    price_lst = []
    zipcode_lst = []
    ratings_lst = []

    notadded_lst = []

    keys_lst = ['name', 'categories', 'coordinates', 'price', 'location']

    for idx, bus_dict in enumerate(businesses):
        bus_keys_lst = list(bus_dict.keys())
        if 'name' in bus_keys_lst and 'categories' in bus_keys_lst and 'coordinates' in bus_keys_lst and 'price' in bus_keys_lst and 'location' in bus_keys_lst and 'rating' in bus_keys_lst:        
            rest_names_lst.append(bus_dict['name'])
            cats_dict = bus_dict['categories'][0]
            category_lst.append(cats_dict['title'])
            coordinates_dict = bus_dict['coordinates']
            lat_lst.append(coordinates_dict['latitude'])
            lon_lst.append(coordinates_dict['longitude'])
            price_lst.append(bus_dict['price'])
            location_dict = bus_dict['location']
            zipcode_lst.append(location_dict['zip_code'])
            ratings_lst.append(bus_dict['rating'])
        elif 'name' in bus_keys_lst and 'categories' in bus_keys_lst and 'coordinates' in bus_keys_lst and 'rating' in bus_keys_lst:
            rest_names_lst.append(bus_dict['name'])
            cats_dict = bus_dict['categories'][0]
            category_lst.append(cats_dict['title'])
            coordinates_dict = bus_dict['coordinates']
            lat_lst.append(coordinates_dict['latitude'])
            lon_lst.append(coordinates_dict['longitude'])
            ratings_lst.append(bus_dict['rating'])
            price_lst.append(np.NaN)
            zipcode_lst.append(np.NaN)

        elif 'name' in bus_keys_lst and 'categories' in bus_keys_lst and 'coordinates' in bus_keys_lst and 'price' in bus_keys_lst:
            rest_names_lst.append(bus_dict['name'])
            cats_dict = bus_dict['categories'][0]
            category_lst.append(cats_dict['title'])
            coordinates_dict = bus_dict['coordinates']
            lat_lst.append(coordinates_dict['latitude'])
            lon_lst.append(coordinates_dict['longitude'])
            price_lst.append(bus_dict['price'])
            zipcode_lst.append(np.NaN)
            ratings_lst.append(np.NaN)

        elif 'name' in bus_keys_lst and 'categories' in bus_keys_lst and 'coordinates' in bus_keys_lst:
            rest_names_lst.append(bus_dict['name'])
            cats_dict = bus_dict['categories'][0]
            category_lst.append(cats_dict['title'])
            coordinates_dict = bus_dict['coordinates']
            lat_lst.append(coordinates_dict['latitude'])
            lon_lst.append(coordinates_dict['longitude'])
            price_lst.append(np.NaN)
            zipcode_lst.append(np.NaN)
            ratings_lst.append(np.NaN)

    rest_df = pd.DataFrame({'Name': rest_names_lst, 'Category': category_lst, 'Latitude': lat_lst, 'Longitude': lon_lst, 'Price': price_lst, 'Rating': ratings_lst, 'Zipcode': zipcode_lst})
    dfs_lst.append(rest_df)
    offset += 50
    

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

In [30]:
# Save to csv 

all_rests_df = pd.concat(dfs_lst)
len(all_rests_df)
len(all_rests_df['Name'].unique())

all_rests_df.to_csv('/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/DataViz_projectdata/yelp_rest_data.csv', index = False)